<a href="https://colab.research.google.com/github/sgr1118/deep-learning-from-scratch-3/blob/main/%EC%A0%9C1%EA%B3%A0%EC%A7%80_%EB%AF%B8%EB%B6%84_%EC%9E%90%EB%8F%99_%EA%B3%84%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목표
- 총 10단계로 구성된 미분을 자동으로 계산하는 틀을 만든다.
- 이번 고지에서는 미분을 자동으로 계산하기 위해 '변수'와 '함수'를 표현하는 두 클래스 Vairable과 Function을 만든다.

## 1단계 : 상자로서의 변수

### 1.1 변수란
- 데이터가 할당되어지는 것, 비유하자면 변수는 상자이고 데이터를 그 안에 넣는것이다.

### 1.2 Variable 클래스 구현


In [ ]:
class Variable:
    def __init__(self, data): # __init__ 초기화 함수에 주어진 인수를 인스턴스 변수 data에 대입
        self.data = data

In [ ]:
import numpy as np

data = np.array(1.0) # 넘파이의 다차원 배열 사용
x = Variable(data)
print(x.data)

1.0


In [ ]:
x.data = np.array(2.0)
print(x.data)

2.0


### 1.3 넘파이의 다차원 배열
- 다차원 배열은 숫자 등의 원소가 일정하게 모여 있는 데이터 구조
- 다차원 배열에서 우너소의 순서에는 방향이 있고, 이 방향을 차원(dimension) or 축(axis)이라고 한다.
- 0차원 : 스칼라, 1차원 : 벡터, 2차원 : 행렬

In [ ]:
# 다차원 배열 예시
import numpy as np

x = np.array(1)
print(x.ndim) # 결과는 스칼라

y = np.array([1,2,3])
print(y.ndim) # 결과는 벡터

z = np.array([[1,2,3],
             [4,5,6]])
print(z.ndim) # 결과는 행렬

0
1
2


## 2단계 : 변수를 낳는 함수

### 2.1 함수란
- 변수 사이의 대응 관계를 정하는 역할을 함수라고 한다. 예시로 $f(x) - x^2$이고 $y = f(x)$라고 하면 변수 y와 x의 관계가 함수$f$에 의해 결정된다.

### 2.2 Function 클래스 구현
- Function 클래스는 Variable 인스턴스를 입력받아 Variable 인스터스를 출력
- Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 있다.

In [ ]:
class Function:
    def __call__(self, input): # __call__ 메서드의 인수 input은 Variable 인스턴스라고 가정
        x = input.data # 데이터를 꺼낸다
        y = x ** 2 # 실제 계산
        output = Variable(y) # Variable 형태로 되돌림
        return output

### 2.3 Function 클래스 이용

In [ ]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


- 현재 Function 클래스는 '입력값의 제곱'으로 고정된 함수이지만 모든 계산을 할 수 있어야하기에 forward 메서드를 사용할 거이다. 다음 두 가지 사항을 충족하도록 구현한다.

- Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현
- 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현

In [ ]:
# 수정
class Function:
    def __call__(self, input): # __call__ 메서드의 인수 input은 Variable 인스턴스라고 가정
        x = input.data # 데이터를 꺼낸다
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y) # Variable 형태로 되돌림
        return output

    def forward(self, x):
        raise NotImplementedError() 

# Function 클래스의 forward 메서드는 예외를 발생시킨다. 이렇게 해두면 Function 클래스의 forward 메서드를
# 직접 호출한 사람에데 '이 메서드는 상속하여 구현해야 한다'는 사실을 알려줄 수 있다.

In [ ]:
class Square(Function):
    def forward(self, x):
        return x ** 2

- Square 클래스는 Function 클래스를 상속하기에 __call__ 메서드는 그대로 계승

In [ ]:
# 결과 확인

x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## 3단계 : 함수 연결

### 3.1 Exp 함수 구현
- $y = e^x$라는 오일러의 수 또는 네이피어 상수라고 불리는 것을 구현

In [ ]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x) # 계산 값

### 3.2 함수 연결

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b) # 3개의 함수를 연이어 동작하게 했다. 이처럼 여러 함수로 구성된 함수를 합성 함수라 한다.

print(y.data)

1.648721270700128


## 4단계 : 수치 미분

### 4.1 미분이란
$$f^\prime (x) = lim_{h > 0} \frac{f(x+h) - f(x)}{h}$$
- '극한으로 짧은 시간(순간)'에서의 변화량이다.
- $f^\prime (x)도 함수이며 f(x)$의 도함수이다.

### 4.2 수치 미분 구현
- 컴퓨터는 극한을 취급할 수 없어서 h를 극한과 비슷한 값 h = 0.0001과 같은 매우 작은 값을 이용한다. 이런 미세한 차이를 이용하여 함수의 변화령을 구하는 방법을 수치 미분이라 한다.

In [ ]:
def numerical_diff(f, x, eps = 1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps) # 중앙 차분의 직선의 기울기

In [ ]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy) # 함수 y = x^2에서 x = 2.0일 때 수치 미분한 결과

4.000000000004


### 4.3 함성 함수의 미분
$y = ({e^{x}}^{2})^2$이라는 계산을 미분

In [ ]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


### 4.4 수치 미분의 문제점
- 수치 미분에는 오차가 포함된다. 어떤 계산이냐에 따라 커질 수도 작을 수도 있다.
- 수치 미분의 더 심각한 문제는 계산량이 많다. 이런 문제를 해결하기 위해 역전파를 사용한다.

## 5단계 : 역전파 이론

### 5.1 연쇄 법칙
- 역전파를 이해하기 위한 이론은 연쇄 법칙이다. 연쇄 법칙에 따르면 합성 함수의 미분은 구성 함수 각각을 미분한 후 곱한 것과 같다. 아래는 예시

![](https://blog.kakaocdn.net/dn/9U9J5/btqIvDCbsVv/mS9ZrimZQoD2Ghzu85HIcK/img.png)
<center>역전파 그림</center>

- ML은 주로 대량의 매개변수를 입력받아서 마지막에 손실 함수를 거쳐 출력을 내는 형태로 진행됩니다. 손실 함수의 출력은 단일한 스칼라값이며, 이 값이 중요 요소 이다.
- 즉 손실 함수의 각 매개변수에 대한 미분을 계산해야 한다.
- 이런 경우 미분값을 출력에서 입력 방향으로 전파하면 한 번의 전파만으로 모든 매개변수에 대한 미분을 계산할 수 있다. 이처럼 효율적으로 이뤄지기 때문에 미분 역전파를 이용하는 것이다.

## 6단계 : 수동 역전파

### 6.1 Variable 클래스 추가 구현

In [ ]:
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None # 미분값 저장

## 6.2 Function 클래스 추가 구현
- 이전 단계에서는 순전파 기능만 있었지만 아래 두 기능을 추가한다.
- 미분을 계산하는 역전파 (backward 메서드), forward 메서드 호출 시 건네받은 Variable 인스턴스 유지

In [ ]:
class Function:
    def __call__(self, input): # __call__ 메서드의 인수 input은 Variable 인스턴스라고 가정
        x = input.data # 데이터를 꺼낸다
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y) # Variable 형태로 되돌림
        self.input = input # 입력 변수를 기억한다.
        return output

    def forward(self, x):
        raise NotImplementedError() 

    def backward(self, gy):
        raise NotImplementedError()

### 6.3 Square와 Exp 클래스 추가 구현

In [ ]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy # y = x^2의 미분은 2 * x
        return gx

In [ ]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x) # 계산 값
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

### 6.4 역전파 구현

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

- 이어서 역전파를 y로 미분한다.

In [ ]:
y.grad = np.array(1.0) # 역전파는 dy/dy = 1에서 시작
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad) # 4단계 수치 미분에서 구한 값과 유사하다.

3.297442541400256


## 7단계 : 역전파 자동화
- 역전파를 더 효율적으로 사용하기 위한 과정
- 순전파를 한 번만 해주면 어떤 계산이라도 상관없이 역전파가 자동으로 이루어지는 구조를 만들 것이다. 이것을 Define-by-Run(동적 계산 그래프)라고 한다.

### 7.1 역전파 자동화의 시작
- 함수 관점에서 변수는 '입력'과 '출력'에 쓰인다. 즉 입력 변수와 출력 변수로서 존재
- 변수 관점에서 변수는 함수에 의해 만들어진다. 즉 변수에게 있어 함수는 창조자이다.

In [ ]:
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가

    def set_creator(self, func): # 메서드 추가
        self.creator = func

In [ ]:
class Function:
    def __call__(self, input): # __call__ 메서드의 인수 input은 Variable 인스턴스라고 가정
        x = input.data # 데이터를 꺼낸다
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y) # Variable 형태로 되돌림
        output.set_creator(self) # 출력 변수에 창조자를 설정
        self.input = input # 입력 변수를 기억한다.
        self.output = output # 출력 변수를 기억한다.
        return output

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올란다.
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

# 노트북 환경에서는 실행되지않고 py로 실행 가능

- assert문이란 '주장하다'라는 의미로 주장에 해당하는 내용을 평가 후 True가 아니라면 예외가 발생한다. 따라서 assert문은 조건을 충족하는지 여부를 확인하는데 사용할 수 있다.

### 7.2 역전파 도전!

In [ ]:
!python step07.py # 역전파 py

2.568050833375483


### 7.3 backward 메서드 추가
- 앞의 과정을 자동화하기 위해 Variable 클래스에 backward라는 메서드 추가

In [ ]:
!python step07.py # 역전파 자동화 py
# 자세한 코드는 ch_1_py 참조

3.297442541400256


## 8단계 : 재귀에서 반복문으로
- 처리 효율을 개선하고 앞으로 확장을 대비해 backward 메서드의 구현 방식 변경

### 8.1 현재의 Variable 클래스
- step07의 backward 메서드에는 입력 방향으로 하나 앞 변수의 backward 메서드를 호출하는 코드이다. 반복적으로 backward 메서드를 호출하는 과정이 반복된다. 이를 재귀 구조라 한다.

### 8.2 반복문을 이용한 구현
- 재귀에서 반복문을 사용한 구현으로 변경한다.

In [ ]:
cd '/content/drive/MyDrive/Book/deeplearng_basic_3/ch_1_py'

/content/drive/MyDrive/Book/deeplearng_basic_3/ch_1_py


In [ ]:
!python step08.py # 역전파 반복문 py
# 자세한 코드는 ch_1_py 참조

3.297442541400256


- 복잡한 계산 그래프를 다룰 때 이런 방법이 처리 효율을 개선 할 수 있다.

## 9단계 : 함수를 더 편리하게
- 앞서 코드에 세 가지 개선을 추가하자

### 9.1 파이썬 함수로 이용하기
- 지금까지는 함수를 '파이썬 클래스'로 정의해서 사용했다. Sequare 클래스를 사용하려면 아래와 같이 작성해야했다.
- 번거로운 과정이기에 '파이썬 함수'를 지원하는 형태로 변경

In [ ]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

In [ ]:
# step09에 추가되는 코드
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

# 간소화
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [ ]:
!python step09.py # 역전파 반복문 파이썬 함수 이용 py
# 자세한 코드는 ch_1_py 참조

3.297442541400256


### 9.2 backward 메서드 간소화
- 두 번째 개선은 y.grad = np.array(1.0) 부분을 생략

In [ ]:
# step09에 추가되는 코드

def backward(self):
    if self.grad is None:
        self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성

In [ ]:
!python step09.py # 역전파 반복문 backward 간소화 py
# 자세한 코드는 ch_1_py 참조

3.297442541400256


### 9.3 ndarray만 취급하기
- Variable은 데이터를 ndarray만 취급하게하였다. 하지만 실수로 float, int를 사용하는 경우도 있으니 Variable 초기화 부분에 아래 코드를 추가한다.

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

In [ ]:
!python step09.py # 역전파 반복문 ndarray만 취급 py
# 자세한 코드는 ch_1_py 참조

Traceback (most recent call last):
  File "/content/drive/MyDrive/Book/deeplearng_basic_3/ch_1_py/step09.py", line 78, in <module>
    y = square(exp(square(x))) # 연속하여 적용
  File "/content/drive/MyDrive/Book/deeplearng_basic_3/ch_1_py/step09.py", line 68, in square
    return Square()(x)
  File "/content/drive/MyDrive/Book/deeplearng_basic_3/ch_1_py/step09.py", line 40, in __call__
    output = Variable(y) # Variable 형태로 되돌림
  File "/content/drive/MyDrive/Book/deeplearng_basic_3/ch_1_py/step09.py", line 15, in __init__
    raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
TypeError: <class 'numpy.float64'>은(는) 지원하지 않습니다.


- 이렇게 수정하면 주의해야 할 사항 예시는 아래와 같다.
- x는 0차원의 ndarray인데, 제곱을 하면 float64가 된다. 다시말하면 DeZero 함수의 게산 결과도 float64, 32가 되는 경우가 발생한다. 이것은 대처가 필요합니다.

In [ ]:
x = np.array(1.0) # 0차원의 ndarray
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [ ]:
# 문제를 해결할 편의 함수
def as_array(x):
    if np.isscalar(x): # 입력데이터 형태 확인
        return np.array(x)
    return x

# 함수 추가후 Function 코드 추가

class Function:
    def __call__(self, input): # __call__ 메서드의 인수 input은 Variable 인스턴스라고 가정
        x = input.data # 데이터를 꺼낸다
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(as_array(y)) # 입력이 스칼라인 경우 ndarray로 변환
        output.set_creator(self) # 출력 변수에 창조자를 설정
        self.input = input # 입력 변수를 기억한다.
        self.output = output # 출력 변수를 기억한다.
        return output

In [ ]:
!python step09.py # 역전파 반복문 ndarray만 취급 py
# 자세한 코드는 ch_1_py 참조

3.297442541400256


## 10단계 : 테스트
- 이번 단계에서는 테스트 방법 특히 딥러닝 프레임워크의 테스트 방법에 대해 알아본다.

### 10.1 파이썬 단위 테스트
- 파이썬으로 테스트할 때는 표준 라이브러리에 포함된 unittest를 사용
- square 함수를 테스트해보기

In [ ]:
# 추가되어지는 코드

import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected) # 주어진 두 객체가 동일한지 여부 판정

In [ ]:
!python -m unittest step10.py

# 1개의 테스트가 수행되었고 결과는 성공

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### 10.2 square 함수의 역전파 테스트

In [ ]:
# 추가된 코드
class SquareTest(unittest.TestCase):
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

In [ ]:
!python -m unittest step10.py

# 마찬가지로 2개의 테스트가 수행되고 성공하였다.

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


### 10.3 기울기 확인을 이용한 자동 테스트
- 앞에서 역전파 테스트를 작성하며 미분의 기댓값을 손으로 계산해 입력하였다. 이 부분을 기울기 확인을 통하여 자동화할 수 있다.
- 기울기 확인이란, 수치 미분으로 구현 결과와 역전파로 구한 결과를 비교하여 그 차이가 크면 역전파 구현에 문제가 있다고 판단하는 검증 기법
- 기울기 확인은 입력값만 있으면 테스트가 가능하기에 효율을 높여준다.

In [ ]:
# 추가된 코드
class SquareTest(unittest.TestCase):
    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 생성
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x) # 수치 미분 계산
        flg = np.allclose(x.grad, num_grad) # 각각 구한 값들이 일치하는지 확인
        self.assertTrue(flg)

In [ ]:
!python -m unittest step10.py

# 마찬가지로 3개의 테스트가 수행되고 성공하였다.

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
